In [1]:
from pandas import DataFrame, read_csv

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
Pandas version 0.18.1
Matplotlib version 1.5.1


In [6]:
df = pd.read_csv('R/spot_for_billy.csv')

In [11]:
df.head(5)

,node,nzdate,tp,price
0,ALB1101,2000-01-01,1,0.34
1,ALB1101,2000-01-01,2,0.31
2,ALB1101,2000-01-01,3,0.31
3,ALB1101,2000-01-01,4,10.11
4,ALB1101,2000-01-01,5,10.15


In [306]:
test = df[(df['node']=='ALB1101') | (df['node']=='ARA2201')]

Looking at only 2 nodes for now, as an example.
Finding the ftr_payoff for the path. 

In [307]:
pivotdf = test.pivot_table(index = ['node', 'nzdate'], columns = 'tp', values = 'price')

So we have the data in wide format, pivoted by the node names and dates. We want to do max(a-b, 0), max(b-a,0) for each trading period for each day. Output will combine the two index nodes as 2 ftr paths with max by tp.

In [308]:
pivotdf.shape

(12424, 50)

In [309]:
ara_spot = pivotdf.loc['ARA2201']

In [310]:
alb_spot = pivotdf.loc['ALB1101']

In [5]:
stu_month_payoffs = pd.read_csv('monthly_payoffs.csv')

In [6]:
stu_month_payoffs.head(5)

,Unnamed: 0,yearmonth,ALB_ARA,ALB_ASB,ALB_ATI,ALB_AVI,ALB_BEN,ALB_BPE,ALB_BRB,ALB_BRK,...,WTU_TWI,WTU_TWZ,WTU_WHI,WTU_WIL,WTU_WKM,WTU_WPA,WTU_WPR,WTU_WRD,WTU_WRK,WTU_WTK
0,1,200001,0.01,0.14,0.04,0.05,0.05,0.04,0.36,0.04,...,0.62,1.83,0.05,0.74,0.15,0.19,NaN,NaN,0.27,1.96
1,2,200002,0.01,0.23,0.21,0.14,0.14,0.06,0.42,0.05,...,2.22,3.14,0.08,1.45,0.36,0.44,NaN,NaN,0.47,3.26
2,3,200003,0.06,1.47,0.15,1.08,1.08,0.08,0.59,0.04,...,3.39,5.24,0.09,1.58,0.50,0.60,NaN,NaN,0.57,5.55
3,4,200004,0.22,0.83,1.86,0.59,0.59,0.55,0.61,0.05,...,2.47,3.90,0.08,2.06,0.40,0.48,NaN,NaN,0.47,4.02
4,5,200005,0.11,1.20,0.23,0.33,0.34,0.44,0.69,0.02,...,5.14,5.91,0.13,3.54,0.49,0.58,NaN,NaN,0.76,6.45


Okay let's look into ALB_ARA and hand calculate out what the payoff for 200001 and 200002 should be. 

In [557]:
alb_spot.index = pd.to_datetime(alb_spot.index)
ara_spot.index = pd.to_datetime(ara_spot.index)

In [569]:
alb_200001 = alb_spot[(alb_spot.index.month == 1) & (alb_spot.index.year == 2000)]
ara_200001 = ara_spot[(ara_spot.index.month == 1) & (ara_spot.index.year == 2000)]

In [576]:
alb_200001.head(5)

tp,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
nzdate,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.34,0.31,0.31,10.11,10.15,10.12,9.93,9.93,9.95,9.95,...,12.27,12.66,15.26,15.26,15.27,15.26,15.27,15.25,NaN,NaN
2000-01-02,12.71,10.36,10.36,10.33,10.33,10.13,10.13,9.98,10.10,10.13,...,15.23,13.81,13.82,13.45,15.26,13.52,15.26,13.02,NaN,NaN
2000-01-03,12.74,12.34,12.72,10.38,10.36,10.36,10.34,10.35,10.35,10.35,...,11.15,13.05,13.75,13.54,13.68,13.66,12.70,12.70,NaN,NaN
2000-01-04,12.93,12.37,12.36,12.44,12.39,10.36,12.31,12.38,12.38,12.31,...,13.38,13.68,13.77,13.74,14.22,14.33,13.59,12.96,NaN,NaN
2000-01-05,12.73,12.25,12.23,10.21,10.35,10.33,10.33,10.33,10.34,10.25,...,14.67,15.32,15.33,15.41,25.02,15.28,15.27,15.28,NaN,NaN


In [577]:
ara_200001.head(5)

tp,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
nzdate,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.33,0.30,0.30,9.77,9.81,9.76,9.59,9.60,9.61,9.61,...,11.85,12.25,14.70,14.70,14.66,14.77,14.85,14.83,NaN,NaN
2000-01-02,12.36,10.09,10.09,10.08,10.08,9.86,9.86,9.71,9.83,9.86,...,14.60,13.24,13.29,12.90,14.78,13.10,14.77,12.61,NaN,NaN
2000-01-03,12.37,12.00,12.38,10.10,10.09,10.09,10.09,10.09,10.09,10.08,...,10.70,12.43,13.09,12.90,13.25,13.23,12.28,12.28,NaN,NaN
2000-01-04,12.54,11.96,11.96,12.03,11.98,10.01,11.88,11.96,11.96,11.89,...,12.85,13.04,13.14,13.10,13.64,13.87,13.12,12.54,NaN,NaN
2000-01-05,12.36,11.92,11.91,9.94,10.09,10.08,10.08,10.08,10.09,9.98,...,13.98,14.47,14.48,14.46,24.07,14.71,14.73,14.71,NaN,NaN


Subtracting `alb_200001` - `ara_200001` will give us the difference per trading period, for all the days in the month

In [580]:
np.maximum(alb_200001 - ara_200001, 0)

tp,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
nzdate,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.01,0.01,0.01,0.34,0.34,0.36,0.34,0.33,0.34,0.34,...,0.42,0.41,0.56,0.56,0.61,0.49,0.42,0.42,NaN,NaN
2000-01-02,0.35,0.27,0.27,0.25,0.25,0.27,0.27,0.27,0.27,0.27,...,0.63,0.57,0.53,0.55,0.48,0.42,0.49,0.41,NaN,NaN
2000-01-03,0.37,0.34,0.34,0.28,0.27,0.27,0.25,0.26,0.26,0.27,...,0.45,0.62,0.66,0.64,0.43,0.43,0.42,0.42,NaN,NaN
2000-01-04,0.39,0.41,0.40,0.41,0.41,0.35,0.43,0.42,0.42,0.42,...,0.53,0.64,0.63,0.64,0.58,0.46,0.47,0.42,NaN,NaN
2000-01-05,0.37,0.33,0.32,0.27,0.26,0.25,0.25,0.25,0.25,0.27,...,0.69,0.85,0.85,0.95,0.95,0.57,0.54,0.57,NaN,NaN
2000-01-06,0.42,0.34,0.34,0.33,0.31,0.31,0.30,0.25,0.31,0.26,...,0.64,0.64,0.71,0.59,0.46,0.46,0.45,0.46,NaN,NaN
2000-01-07,0.46,0.41,0.35,0.35,0.33,0.34,0.33,0.34,0.38,0.35,...,0.47,0.46,0.45,0.83,1.05,0.48,0.75,0.42,NaN,NaN
2000-01-08,0.46,0.44,0.44,0.41,0.40,0.40,0.38,0.34,0.32,0.33,...,0.60,0.62,0.72,0.53,0.79,0.74,0.85,0.40,NaN,NaN
2000-01-09,0.37,0.37,0.35,0.31,0.32,0.31,0.30,0.30,0.08,0.09,...,0.61,0.77,0.72,0.48,0.48,0.74,0.66,0.45,NaN,NaN


Next up is to find the mean per day

In [581]:
np.maximum(alb_200001 - ara_200001, 0).mean(axis=1)

nzdate
2000-01-01     0.400208
2000-01-02     0.438750
2000-01-03     0.477292
2000-01-04     0.539583
2000-01-05     0.760625
2000-01-06     0.584792
2000-01-07     1.812292
2000-01-08     0.620625
2000-01-09     0.478333
2000-01-10     0.984792
2000-01-11     0.768542
2000-01-12     0.923542
2000-01-13     1.060417
2000-01-14     0.846667
2000-01-15     0.888125
2000-01-16     1.118542
2000-01-17     1.553333
2000-01-18    13.027500
2000-01-19     3.319792
2000-01-20     2.121250
2000-01-21     1.448958
2000-01-22     1.660208
2000-01-23     1.648750
2000-01-24     1.556250
2000-01-25     1.305208
2000-01-26     1.196458
2000-01-27     1.046458
2000-01-28     0.891667
2000-01-29     0.459167
2000-01-30     1.128125
2000-01-31     1.255208
dtype: float64

And the last step is to find the mean for the entire month

In [582]:
np.maximum(alb_200001 - ara_200001, 0).mean(axis=1).mean()

1.4942405913978496